In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import jReversion as jR
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score
import networkx as nx
from statannot import add_stat_annotation

In [2]:
networkModel =['bortezomib',
                # 'igvh',
                'apoptosis',
                # 'aurora',
                'bt474_long',
                #'bt474_short',
                # 'cd4t',
                'colitis',
                'death',
                # 'egfr',
                # 'erbb',
                # 'fa_brca',
                # 'fa_check',
                'hcc1954_long',
                #'hcc1954_short',
                'hgf',
                'mammalian',
                # 'mammalian_2006',
                'mapk',
                'oxidative',
                # 'pro_inflammatory',
                #'fibroblasts',
                'skbr3_long',
                #'skbr3_short',
                'tlgl_2008',
                'tlgl_2011',
                # 'tlgl_2011_reduced',
                # 'prostate',
                'migration']

In [3]:
integrated_data = pd.DataFrame(columns=['network', 'num_nodes', 'C0_num', 'del_C0_cycle',
             'C1_num', 'del_C1_cycle', 'C2_num', 'del_C2_cycle',
             'C3_num', 'del_C3_cycle'])

In [4]:
Prefix, Suffix = 'n', 'n'
TEMP = jR.cellcollective(networkModel[0], Prefix, Suffix, directory='../')

BooleanRuleFileName = TEMP['BooleanRule_filename']
network_name = TEMP['network_name']

NumInputs = TEMP['num_inputs']
NumInputConditions = TEMP['num_input_conditions']

InputConditions = TEMP['input_conditions']

OutputNodes = TEMP['output_nodes']
InputNodes = TEMP['input_nodes']

Mapping = TEMP['mapping']
InverseMapping = TEMP['inverse_mapping']
GRead = TEMP['Gread']
ReadNodes = TEMP['read_nodes']

#     CutOffRange = [3, 4, 5, 6, 7]
GRead.remove_edges_from(list(nx.selfloop_edges(GRead)))

# table_original_network = pd.read_csv('../data/' + network_name + '_table_for_original_network.tsv', sep='\t').sort_values(by='node').set_index('node')

# GLabel = nx.DiGraph()
# GLabel.add_edges_from(GRead.edges())
# GLabel = nx.relabel_nodes(GLabel, ReadNodes)


network_data = pd.read_csv('../data/' + network_name + '_table_for_original_network.tsv', sep='\t').sort_values(by='node').set_index('node')
# table_original_network = table_original_network.sort_values(by='node')
# ffl_test = pd.read_csv('../data/' + network_name + '_ffl_test.tsv', sep='\t').sort_values(by='node').set_index('node')
# canalizing_test = pd.read_csv('../data/' + network_name + '_canalizing_check_for_original_network.tsv', sep='\t').sort_values(by='node').set_index('node')


In [5]:
InverseReadNodes = dict()
for key, val in ReadNodes.items():
    InverseReadNodes[val] = key

In [6]:
# C0 only subnetwork
C0 = network_data[network_data['Class'] != 'C0'].index
C0_only_subgraph = nx.subgraph(G=GRead, nbunch=[InverseReadNodes[x] for x in C0])
num_cycle_C0_only_subgrph = len(list(nx.simple_cycles(C0_only_subgraph)))

# C1 only subnetwork
C1 = network_data[network_data['Class'] != 'C1'].index
C1_only_subgraph = nx.subgraph(G=GRead, nbunch=[InverseReadNodes[x] for x in C1])
num_cycle_C1_only_subgrph = len(list(nx.simple_cycles(C1_only_subgraph)))

# C2 only subnetwork
C2 = network_data[network_data['Class'] != 'C2'].index
C2_only_subgraph = nx.subgraph(G=GRead, nbunch=[InverseReadNodes[x] for x in C2])
num_cycle_C2_only_subgrph = len(list(nx.simple_cycles(C2_only_subgraph)))

# C3 only subnetwork
C3 = network_data[network_data['Class'] != 'C3'].index
C3_only_subgraph = nx.subgraph(G=GRead, nbunch=[InverseReadNodes[x] for x in C3])
num_cycle_C3_only_subgrph = len(list(nx.simple_cycles(C3_only_subgraph)))

In [7]:
cycle_data = {'network': network_name, 'num_nodes': len(GRead), 'C0_num': len(C0), 'del_C0_cycle': num_cycle_C0_only_subgrph,
             'C1_num': len(C1), 'del_C1_cycle': num_cycle_C1_only_subgrph, 'C2_num': len(C2), 'del_C2_cycle': num_cycle_C2_only_subgrph,
             'C3_num': len(C3), 'del_C3_cycle': num_cycle_C3_only_subgrph}
integrated_data = integrated_data.append(cycle_data, ignore_index=True)

In [8]:
#cell collective
# networkModel = ['BT474', 'CD4_T', 'death_receptor', 'bortezomib', 'lymphoid']

for Model in networkModel[1:]:
    Prefix, Suffix = 'n', 'n'
    TEMP = jR.cellcollective(Model, Prefix, Suffix, directory='../')

    BooleanRuleFileName = TEMP['BooleanRule_filename']
    network_name = TEMP['network_name']

    NumInputs = TEMP['num_inputs']
    NumInputConditions = TEMP['num_input_conditions']

    InputConditions = TEMP['input_conditions']

    OutputNodes = TEMP['output_nodes']
    InputNodes = TEMP['input_nodes']
    
    Mapping = TEMP['mapping']
    InverseMapping = TEMP['inverse_mapping']
    GRead = TEMP['Gread']
    ReadNodes = TEMP['read_nodes']
    
#     CutOffRange = [3, 4, 5, 6, 7]
    GRead.remove_edges_from(list(nx.selfloop_edges(GRead)))
    
    InverseReadNodes = dict()
    for key, val in ReadNodes.items():
        InverseReadNodes[val] = key

    network_data = pd.read_csv('../data/' + network_name + '_table_for_original_network.tsv', sep='\t').sort_values(by='node').set_index('node')
    
    C0 = network_data[network_data['Class'] != 'C0'].index
    C0_only_subgraph = nx.subgraph(G=GRead, nbunch=[InverseReadNodes[x] for x in C0])
    num_cycle_C0_only_subgrph = len(list(nx.simple_cycles(C0_only_subgraph)))

    # C1 only subnetwork
    C1 = network_data[network_data['Class'] != 'C1'].index
    C1_only_subgraph = nx.subgraph(G=GRead, nbunch=[InverseReadNodes[x] for x in C1])
    num_cycle_C1_only_subgrph = len(list(nx.simple_cycles(C1_only_subgraph)))

    # C2 only subnetwork
    C2 = network_data[network_data['Class'] != 'C2'].index
    C2_only_subgraph = nx.subgraph(G=GRead, nbunch=[InverseReadNodes[x] for x in C2])
    num_cycle_C2_only_subgrph = len(list(nx.simple_cycles(C2_only_subgraph)))

    # C3 only subnetwork
    C3 = network_data[network_data['Class'] != 'C3'].index
    C3_only_subgraph = nx.subgraph(G=GRead, nbunch=[InverseReadNodes[x] for x in C3])
    num_cycle_C3_only_subgrph = len(list(nx.simple_cycles(C3_only_subgraph)))
    
    cycle_data = {'network': network_name, 'num_nodes': len(GRead), 'C0_num': len(C0), 'del_C0_cycle': num_cycle_C0_only_subgrph,
             'C1_num': len(C1), 'del_C1_cycle': num_cycle_C1_only_subgrph, 'C2_num': len(C2), 'del_C2_cycle': num_cycle_C2_only_subgrph,
             'C3_num': len(C3), 'del_C3_cycle': num_cycle_C3_only_subgrph}
    integrated_data = integrated_data.append(cycle_data, ignore_index=True)

In [9]:
integrated_data.to_csv('../class_cycle_num_cellCollective_2.csv')